In [1]:
ls

0_To generate n-best hypothesis.ipynb    torgo.csv
1_To generate audio_features.ipynb       torgo_test_single.ipynb
2_finetune_whisper_on_torgo-Copy1.ipynb  Untitled.ipynb
3_prepare_for_torgo_baseline.ipynb       whisper-tiny/
output/


In [1]:
pwd

'/work/van-speech-nlp/jindaznb/jslpnb/mllm_expriments/Whispering-LLaMA/data_preparation'

In [25]:
!sbatch tinf.bash

Submitted batch job 42890852


In [22]:
!sbatch ttrain.bash

Submitted batch job 42889119


In [8]:
!sbatch tdata.bash

Submitted batch job 42795999


In [29]:
!squeue -u zhang.jinda1

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          42884605       gpu ttrain.b zhang.ji  R    4:34:41      1 d1029
          42884604       gpu ttrain.b zhang.ji  R    5:02:42      1 d1028
          42890852       gpu tinf.bas zhang.ji  R      36:15      1 d1004
          42869025     short sys/dash zhang.ji  R   14:58:11      1 c0185


In [30]:
!scancel 42890852

In [2]:
import json
with open('training_args.json') as training_args_file:
    training_args_dict = json.load(training_args_file)

# Replace the default values with the values from the command line arguments
training_args_dict['learning_rate'] = learning_rate
training_args_dict['per_device_train_batch_size'] = train_batch_size
training_args_dict['per_device_eval_batch_size'] = eval_batch_size
training_args_dict['seed'] = seed
training_args_dict['gradient_accumulation_steps'] = gradient_accumulation_steps
training_args_dict['optim'] = optimizer
training_args_dict['lr_scheduler_type'] = lr_scheduler_type
training_args_dict['num_train_epochs'] = num_epochs

training_args = TrainingArguments(
    output_dir=model_local_path,
    hub_model_id=repo_name,
    **training_args_dict
)

NameError: name 'learning_rate' is not defined

In [6]:
import os
import glob

def delete_files_with_prefix(prefix):
    # Get the current working directory
    current_dir = os.getcwd()

    # Create a search pattern for files starting with the given prefix
    search_pattern = os.path.join(current_dir, f"{prefix}*")

    # Find all files matching the search pattern
    files_to_delete = glob.glob(search_pattern)

    # Iterate over the files and delete them
    for file_path in files_to_delete:
        try:
            os.remove(file_path)
            print(f"Deleted: {file_path}")
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")

# Example usage
prefix = "slurm-4271395"  # Replace with your actual prefix
delete_files_with_prefix(prefix)

Deleted: /work/van-speech-nlp/jindaznb/jslpnb/mllm_expriments/Whispering-LLaMA/data_preparation/slurm-42713952.out


In [5]:
import json
with open('training_args.json') as training_args_file:
    training_args_dict = json.load(training_args_file)

In [6]:
training_args_dict

{'evaluation_strategy': 'steps',
 'per_device_train_batch_size': 4,
 'per_device_eval_batch_size': 4,
 'gradient_accumulation_steps': 2,
 'eval_delay': 0,
 'learning_rate': 0.0001,
 'weight_decay': 0.005,
 'adam_beta1': 0.9,
 'adam_beta2': 0.999,
 'adam_epsilon': 1e-08,
 'max_grad_norm': 1.0,
 'max_steps': -1,
 'lr_scheduler_type': 'linear',
 'warmup_ratio': 0.0,
 'warmup_steps': 1000,
 'save_strategy': 'steps',
 'save_steps': 500,
 'save_total_limit': 3,
 'report_to': 'all',
 'seed': 42,
 'eval_steps': 1000,
 'num_train_epochs': 20,
 'optim': 'adamw_torch',
 'optim_args': None,
 'adafactor': False,
 'group_by_length': True,
 'length_column_name': 'length',
 'push_to_hub': True,
 'hub_strategy': 'every_save'}

In [1]:
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration

# Load the tokenizer, retriever, and model
tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-nq")
retriever = RagRetriever.from_pretrained("facebook/rag-token-nq", use_dummy_dataset=True)
model = RagSequenceForGeneration.from_pretrained("facebook/rag-token-nq", retriever=retriever)

# Define the input query
input_query = "What is the capital of France?"

# Tokenize the input query
input_ids = tokenizer(input_query, return_tensors="pt").input_ids

# Generate the response
output_ids = model.generate(input_ids)

# Decode the generated response
response = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0]

print("Query:", input_query)
print("Response:", response)

/work/van-speech-nlp/jindaznb/mmenv/lib/python3.9/site-packages/transformers/models/bart/configuration_bart.py:179: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizerFast'.


ImportError: 
RagRetriever requires the faiss library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/facebookresearch/faiss/blob/master/INSTALL.md and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [10]:
import torch
!cd ..
data_path = "Inference/gs_inferences/torgo_F01_test.pt"
data   = torch.load(data_path,map_location=torch.device('cpu'))

In [9]:
cd ..

/work/van-speech-nlp/jindaznb/jslpnb/mllm_expriments/Whispering-LLaMA


In [11]:
len(data)

460

In [12]:
import torch
!cd ..
data_path = "Inference/gs_inferences/torgo_F01_val.pt"
data   = torch.load(data_path,map_location=torch.device('cpu'))

len(data)

460

In [ ]:
!speaker_id='F01' && python training/WL-S-Copy1.py --lr 1e-3 \
    --d 1 \
    --pretrained_path 'weights/alpaca.pth' \
    --tokenizer_path 'weights/tokenizer.model' \
    --data 'Inference/gs_inferences/torgo_${speaker_id}'

In [3]:
from datasets import load_dataset
from pprint import pprint

# Load the HyPoradise-v1-GigaSpeech dataset
dataset = load_dataset("PeacefulData/HyPoradise-v1-GigaSpeech")

# Example: Print the first example from the train split
pprint(dataset['train'][0])

{'ID': 'YOU0000000145_S0000030',
 'Unnamed: 0': 0,
 'audio_time': 1.67999267578125,
 'category': 25,
 'category_name': 'Entertainment',
 'ground_truth': 'so close to my mouth.',
 'hypothesis_1': 'its so close to my mouth',
 'hypothesis_10': 'socialist tomorrow',
 'hypothesis_11': 'suitable so close to my mouth',
 'hypothesis_12': 'sılcostzamerio',
 'hypothesis_13': 'thats so close to my mouth',
 'hypothesis_14': None,
 'hypothesis_15': None,
 'hypothesis_16': None,
 'hypothesis_17': None,
 'hypothesis_18': None,
 'hypothesis_19': None,
 'hypothesis_2': 'seriously sewing',
 'hypothesis_20': None,
 'hypothesis_3': 'so close to mama',
 'hypothesis_4': 'so close to my mall',
 'hypothesis_5': 'so close to my mouth',
 'hypothesis_6': 'so close to my mouth butt',
 'hypothesis_7': 'so close to my mouth second up',
 'hypothesis_8': 'so close to my mouth!',
 'hypothesis_9': 'so close to your mouth',
 'source': 2}


In [2]:
len(dataset['train'])

32739

In [4]:
len(dataset['test'])

KeyError: 'test'

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'category_name', 'ground_truth', 'hypothesis_1', 'hypothesis_2', 'hypothesis_3', 'hypothesis_4', 'hypothesis_5', 'hypothesis_6', 'hypothesis_7', 'hypothesis_8', 'hypothesis_9', 'hypothesis_10', 'hypothesis_11', 'hypothesis_12', 'hypothesis_13', 'hypothesis_14', 'hypothesis_15', 'hypothesis_16', 'hypothesis_17', 'hypothesis_18', 'hypothesis_19', 'hypothesis_20', 'source', 'category', 'audio_time', 'ID'],
        num_rows: 32739
    })
})

In [14]:
import base64
import numpy as np

_ALIGNMENT_HEADS = {
    "tiny.en": b"ABzY8J1N>@0{>%R00Bk>$p{7v037`oCl~+#00",
    # Add other models as needed...
}

def get_alignment_heads(model_key):
    if model_key not in _ALIGNMENT_HEADS:
        raise ValueError("Model key not found in alignment heads dictionary")
    
    encoded_string = _ALIGNMENT_HEADS[model_key]
    # Decode the base85 string
    decoded_bytes = base64.b85decode(encoded_string)
    # Convert bytes to boolean array
    bool_array = np.unpackbits(np.frombuffer(decoded_bytes, dtype=np.uint8)).astype(bool)
    
    return bool_array

# Example usage
model_key = "tiny.en"
alignment_heads = get_alignment_heads(model_key)
print(alignment_heads)

[False False False  True  True  True  True  True  True False False False
  True False  True  True False False False False  True False False False
 False False False False False False False False False False  True  True
  True False  True  True False False  True False  True False False  True
  True  True False  True  True False  True False False  True  True False
 False False  True  True False False False False False False  True False
  True  True  True  True  True  True  True  True False  True  True False
 False False  True  True False  True  True False False False False False
 False False False False False False False False False False False False
 False False False  True False  True False False False  True  True False
 False False False False False  True False False  True  True False False
  True False False  True False False False False  True False False False
 False False False False False  True  True False False False False False
  True  True False False False False False False Fa

In [28]:
a = "Hello, "
b = None

result = a + b  # This will raise a TypeError

TypeError: can only concatenate str (not "NoneType") to str

In [ ]:
result